In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

In [4]:
from pyspark.sql.functions import current_timestamp

In [5]:
spark = SparkSession \
    .builder \
    .appName("Bespot app") \
    .config("spark.executor.extraClassPath", "/path/to/postgresql-42.6.0.jar") \
    .config("spark.jars", "/path/to/postgresql-42.6.0.jar") \
    .getOrCreate()

23/05/26 20:59:50 WARN Utils: Your hostname, alexandros-OptiPlex-3010 resolves to a loopback address: 127.0.1.1; using 192.168.1.17 instead (on interface enp2s0)
23/05/26 20:59:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
23/05/26 20:59:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/26 20:59:55 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [6]:
dataset1_schema = StructType(fields=[StructField("user", StringType(), False),
                                     StructField("session_id", StringType(), False),
                                     StructField("timestamp", IntegerType(), False),
                                     StructField("page", StringType(), False),
])

In [7]:
dataset2_schema = StructType(fields=[StructField("user", StringType(), False),
                                     StructField("session_id", StringType(), False),
                                     StructField("timestamp", IntegerType(), False),
                                     StructField("transaction", StringType(), False),
])

In [8]:

df_dataset1 = spark.read.option("header", True).schema(dataset1_schema).csv(r"dataset1.csv")
df_dataset1 = df_dataset1.withColumnRenamed("user", "user_id") \
                        .withColumn("created_on", current_timestamp())


In [9]:
df_dataset2 = spark.read.option("header", True).schema(dataset2_schema).csv(r"dataset2.csv")
df_dataset2 = df_dataset2.withColumnRenamed("user", "user_id") \
                        .withColumn("created_on", current_timestamp())


In [10]:
url = "jdbc:postgresql://localhost:5432/bespot1"

properties = {
    "user": "myprojectuser",
    "password": "password",
    "driver": "org.postgresql.Driver"
}

mode = "append"

In [11]:
df_dataset1.write.jdbc(url=url, table='user_pages_process', mode=mode, properties=properties)

In [12]:
df_dataset2.write.jdbc(url=url, table='user_trans_process', mode=mode, properties=properties)